In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pickle

In [2]:
# Functions
def setHeaders(dataSet):
    dataSet.columns = ['stars','title','description']
    dataSet['fake'] = 0
    dataSet.to_csv('tain_example.csv', index=False)

In [3]:
# refactor fake reviews
# df = pd.read_csv('fake_reviews_dataset.csv')

# # Step 2: Define the new column order
# new_column_order = ['rating','label','text_']  # Specify your desired column order

# # Step 3: Reorder columns
# df = df[new_column_order]

# df.drop('category', axis=1, inplace=True)
# df['fake'] = 1
# Step 4: Write to CSV
# df.to_csv('fake_reviews_dataset.csv', index=False) 

# bad_reviews = pd.read_csv('fake_reviews_dataset.csv')
# print(bad_reviews.head())

In [4]:
# Split the data
real_reviews_dataSet = pd.read_csv('tain_example.csv')
fake_reviews_dataSet = pd.read_csv('fake_reviews_dataset.csv')

# Split the data
X = real_reviews_dataSet.iloc[:100, 2].values  
# print(fake_reviews_dataSet.iloc[:100, 2].values )
X = pd.concat([pd.Series(X), pd.Series(fake_reviews_dataSet.iloc[:100, 2].values)], axis=0).values

y = real_reviews_dataSet['fake'].iloc[:100].values  # Assuming 'fake' column contains labels (0 for genuine, 1 for fake)
y = pd.concat([pd.Series(y),pd.Series(fake_reviews_dataSet['fake'].iloc[:100].values)], axis=0).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
# Vectorize
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # Adjust max_features as needed

vectored_x_train = tfidf_vectorizer.fit_transform(X_train)
vectored_x_test = tfidf_vectorizer.transform(X_test)

# save vectorizer
with open('fake_or_vectorizer.pkl','wb') as f:
    pickle.dump(tfidf_vectorizer,f)



In [6]:
# Train the model
classifier = LogisticRegression()
classifier.fit(vectored_x_train, y_train)

# save model
with open('fake_or_not.pkl','wb') as f:
    pickle.dump(classifier,f)

LogisticRegression()

In [8]:
predictionStatement_real = 'I heard about this book, so I wanted to read it to see if it was as funny and stupid (unintentionally) as I heard.I was not disappointed. Few things in life are free, and sex is defintely not among them.The perfect coda to this book is that the couple got divroced several years after publishing this book. You can\'t make this stuff up.'
predictions = classifier.predict(tfidf_vectorizer.transform([predictionStatement_real]))
print('Fake?'+str(predictions))

predictionStatement_fake = 'Love this!  Well made, sturdy, and very comfortable.  I love it!Very pretty'
predictions = classifier.predict(tfidf_vectorizer.transform([predictionStatement_fake]))
print('Fake?'+str(predictions))


Fake?[0]
Fake?[1]
